# GFK

In [1]:
%load_ext autoreload
%autoreload 2

In [38]:
#adaptacion
from utils.adaptacion import gfk_train_all, transform_gfk, gfk_compute_accuracy

#clasificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from utils.clasificacion import *

#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#otros
import os
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import itertools

#variables para guardar los resultados
tipo = pruebas[2]
dataset_name = datasets[0]
dims = dimensions[dataset_name]
marco = 1

## Pruebas con el dataset Amazon

In [3]:
print tipo
print dataset_name
print dims
print data_path

gfk
amazon
3000
data


In [30]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
unlabeled = dataset_object.unlabeled
domains = dataset_object.domains

Dataset already splitted


# Adaptación

## Creación de modelos de adaptación.

Para cada dominio se entrenan distintos modelos según los parámetros enviados.

Cada modelo es guardado en la ruta: models/amazon/gfk/me1\_[dominio\_objetivo]_[numero_de_modelo].pkl

Todas las rutas son guardadas en un diccionario separado por dominios, en donde cada dominio contiene una lista con las rutas. El diccionario es almacenado en: models/amazon/gfk/me1_models_paths.pkl


In [32]:
def get_source_target_train_data(target_domain, dataset_object):
    X = None
    X_tgt = None
    for domain in dataset_object.domains:
        if target_domain == domain:
            X_tgt = dataset_object.get_all_domain_X(target_domain)
        else:
            if X is None:
                X = dataset_object.get_all_domain_X(domain)
            else:
                X_tmp = dataset_object.get_all_domain_X(domain)
                X = np.concatenate([X, X_tmp])
    
    return X, X_tgt


X, X_tgt = get_source_target_train_data('books', dataset_object)

(21212, 3000)
(6465, 3000)


In [34]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me1_models_paths.pkl')
paths_dict = {}

# si existe el archivo con las rutas
# se carga la lista con las rutas
if os.path.exists(models_paths):    
    print "Cargando rutas de modelos adaptados."
    paths_dict = joblib.load(models_paths)
    print "Rutas cargadas en la variable 'paths_dict'"

# si no
# se entrenan los modelos y se obtiene la lista con rutas
else:
    #se establecen los parametros para los modelos
    parameters = {
        'dims': [int(dims/4), int(dims/2)],
        'n_subs': [10, 20, 50]
    }
    
    print "Creando modelos de adaptacion..."
    for target_domain in domains:
        print target_domain
        X , X_tgt = get_source_target_train_data(target_domain, dataset_object)
        
        folder_path = os.path.join(models_path, dataset_name, tipo)
        prefix = "me1_"+target_domain+"_"
        
        new_paths = gfk_train_all(X, X_tgt, parameters, folder_path, prefix)
    
        paths_dict[target_domain] = new_paths
    
    print "\nCreacion de modelos terminada\nGuardando rutas en %s" % models_paths
    joblib.dump(paths_dict, models_paths)
    print "Rutas cargadas en la variable 'paths_dict'"

Creando modelos de adaptacion...
electronics
	Entrenando modelo 0
	Guardando modelo en models/amazon/gfk/me1_electronics_0.pkl
	Entrenando modelo 1
	Guardando modelo en models/amazon/gfk/me1_electronics_1.pkl
	Entrenando modelo 2
	Guardando modelo en models/amazon/gfk/me1_electronics_2.pkl
	Entrenando modelo 3
	Guardando modelo en models/amazon/gfk/me1_electronics_3.pkl
	Entrenando modelo 4
	Guardando modelo en models/amazon/gfk/me1_electronics_4.pkl
	Entrenando modelo 5
	Guardando modelo en models/amazon/gfk/me1_electronics_5.pkl
dvd
	Entrenando modelo 0
	Guardando modelo en models/amazon/gfk/me1_dvd_0.pkl
	Entrenando modelo 1
	Guardando modelo en models/amazon/gfk/me1_dvd_1.pkl
	Entrenando modelo 2
	Guardando modelo en models/amazon/gfk/me1_dvd_2.pkl
	Entrenando modelo 3
	Guardando modelo en models/amazon/gfk/me1_dvd_3.pkl
	Entrenando modelo 4
	Guardando modelo en models/amazon/gfk/me1_dvd_4.pkl
	Entrenando modelo 5
	Guardando modelo en models/amazon/gfk/me1_dvd_5.pkl
kitchen
	Entren

In [35]:
print "Modelos almacenados en:"
for dominio, rutas in paths_dict.items():
    print dominio
    for ruta in rutas:
        print "\t", ruta     

Modelos almacenados en:
dvd
	models/amazon/gfk/me1_dvd_0.pkl
	models/amazon/gfk/me1_dvd_1.pkl
	models/amazon/gfk/me1_dvd_2.pkl
	models/amazon/gfk/me1_dvd_3.pkl
	models/amazon/gfk/me1_dvd_4.pkl
	models/amazon/gfk/me1_dvd_5.pkl
electronics
	models/amazon/gfk/me1_electronics_0.pkl
	models/amazon/gfk/me1_electronics_1.pkl
	models/amazon/gfk/me1_electronics_2.pkl
	models/amazon/gfk/me1_electronics_3.pkl
	models/amazon/gfk/me1_electronics_4.pkl
	models/amazon/gfk/me1_electronics_5.pkl
books
	models/amazon/gfk/me1_books_0.pkl
	models/amazon/gfk/me1_books_1.pkl
	models/amazon/gfk/me1_books_2.pkl
	models/amazon/gfk/me1_books_3.pkl
	models/amazon/gfk/me1_books_4.pkl
	models/amazon/gfk/me1_books_5.pkl
kitchen
	models/amazon/gfk/me1_kitchen_0.pkl
	models/amazon/gfk/me1_kitchen_1.pkl
	models/amazon/gfk/me1_kitchen_2.pkl
	models/amazon/gfk/me1_kitchen_3.pkl
	models/amazon/gfk/me1_kitchen_4.pkl
	models/amazon/gfk/me1_kitchen_5.pkl


## Busqueda del mejor modelo por dominio

Por cada dominio se busca el mejor modelo de adaptación.

Esto se obtiene adaptando los datos de entrenamiento de cada dominio por cada modelo creado y realizando Grid-Search y Cross-Validation con estos datos.

El modelo que logre un mejor valor de Cross-Validation es considerado el mejor modelo para adaptar y queda guardado en un diccionario de la forma:

```python
best_models = {
    [dominio_1] = ruta,
    .
    .
    .
    [dominio_n] = ruta,
}
```

Este diccionario queda almacenado en la ruta:
    models/amazon/gfk/best_models.pkl

In [46]:
best_models_paths = os.path.join(models_path, dataset_name, tipo, "me1_best_models.pkl")
best_models = {}

if os.path.exists(best_models_paths):
    print "Cargando rutas de los mejores modelos..."
    best_models = joblib.load(best_models_paths)
    print "Rutas cargadas"
else:
    print "Obteniendo mejores modelos..."
    pairs = list(itertools.permutations(domains, 2))
    
    for src, tgt in pairs:
        print "src: %s - tgt: %s" % (src, tgt)
        rutas = paths_dict[tgt]
        
         # se obtienen los datos del dominio
        X_tr = np.asarray(labeled[src]['X_tr'].todense())
        y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
        
        mejor_score = 0
        best_path = None
        for ruta in rutas:
            gfk = joblib.load(ruta)
            X_deep = transform_gfk(X_tr, gfk)
            
            # se obtiene el mejor score con GridSearch
            new_clf = get_best_score(X_deep, y_tr, classifier='SVC', n_jobs=4)
            new_score = new_clf.best_score_
            
            print "score: %.4f" % new_score
            # se guarda si es el mejor para el modelo
            if new_score > mejor_score:
                mejor_score = new_score
                best_path = ruta
        
        # se guarda la ruta del mejor modelo
        best_models[(src, tgt)] = best_path
    
    # se guarda el diccionario con las mejores rutas
    joblib.dump(best_models, best_models_paths)
    print "Rutas guardadas en ", best_models_paths

Obteniendo mejores modelos...
src: electronics - tgt: dvd
score: 0.8591
score: 0.8623
score: 0.8649
score: 0.8591
score: 0.8623
score: 0.8649
src: electronics - tgt: kitchen
score: 0.8511
score: 0.8698
score: 0.8675
score: 0.8511
score: 0.8702
score: 0.8675
src: electronics - tgt: books
score: 0.8460
score: 0.8608
score: 0.8615
score: 0.8461
score: 0.8608
score: 0.8616
src: dvd - tgt: electronics
score: 0.7714
score: 0.8078
score: 0.8255
score: 0.7719
score: 0.8078
score: 0.8255
src: dvd - tgt: kitchen
score: 0.7715
score: 0.7994
score: 0.8248
score: 0.7715
score: 0.7994
score: 0.8248
src: dvd - tgt: books
score: 0.7770
score: 0.8155
score: 0.8299
score: 0.7770
score: 0.8155
score: 0.8299
src: kitchen - tgt: electronics
score: 0.8605
score: 0.8839
score: 0.8852
score: 0.8605
score: 0.8839
score: 0.8852
src: kitchen - tgt: dvd
score: 0.8633
score: 0.8795
score: 0.8782
score: 0.8633
score: 0.8795
score: 0.8782
src: kitchen - tgt: books
score: 0.8638
score: 0.8816
score: 0.8780
score: 0.8

In [47]:
for dominios, ruta in best_models.items():
    print "%s - %s" % (dominios, ruta)

('dvd', 'electronics') - models/amazon/gfk/me1_electronics_2.pkl
('books', 'kitchen') - models/amazon/gfk/me1_kitchen_2.pkl
('books', 'dvd') - models/amazon/gfk/me1_dvd_2.pkl
('kitchen', 'dvd') - models/amazon/gfk/me1_dvd_1.pkl
('kitchen', 'books') - models/amazon/gfk/me1_books_1.pkl
('books', 'electronics') - models/amazon/gfk/me1_electronics_2.pkl
('electronics', 'dvd') - models/amazon/gfk/me1_dvd_2.pkl
('electronics', 'kitchen') - models/amazon/gfk/me1_kitchen_4.pkl
('kitchen', 'electronics') - models/amazon/gfk/me1_electronics_2.pkl
('dvd', 'kitchen') - models/amazon/gfk/me1_kitchen_2.pkl
('dvd', 'books') - models/amazon/gfk/me1_books_2.pkl
('electronics', 'books') - models/amazon/gfk/me1_books_5.pkl


## Pruebas de clasificación

In [48]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    #se carga el mejor modelo para el par de dominios
    ruta = best_models[(src, tgt)]
    modelo = joblib.load(ruta)

    print "Tarea %d de %d" % (i+1, tareas)

    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%s.pkl" % (tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)

    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)

    #############
    #### GFK ####
    #############
    # se adaptan los dominios usando GFK
    print "Adaptando dominios..."
    X_tr = np.asarray(labeled[src]['X_tr'].todense())
    X_tr_a = transform_gfk(X_tr, modelo)
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
    X_ts_a = transform_gfk(X_ts, modelo)
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se calcula el transfer error
    clf = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
    t_error = 1-clf.score(X_ts_a, y_ts)

    # transfer loss (t)
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['GFK',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]

    i += 1
            
print "\nPruebas completadas."

Tarea 1 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 2 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 3 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 4 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 5 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 6 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 7 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 8 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 9 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 10 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 11 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 12 de 12
Cargando modelo existente.
Adaptando dominios...

Pruebas completadas.


In [49]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,GFK,e->d,electronics,dvd,15.762894,25.478137,9.715243
1,GFK,e->k,electronics,kitchen,8.587715,10.890272,2.302558
2,GFK,e->b,electronics,books,15.157879,25.273132,10.115253
3,GFK,d->e,dvd,electronics,11.127778,18.610465,7.482687
4,GFK,d->k,dvd,kitchen,8.587715,13.232831,4.645116
5,GFK,d->b,dvd,books,15.157879,24.518113,9.360234
6,GFK,k->e,kitchen,electronics,11.127778,14.045351,2.917573
7,GFK,k->d,kitchen,dvd,15.762894,26.888172,11.125278
8,GFK,k->b,kitchen,books,15.157879,31.618290,16.460412
9,GFK,b->e,books,electronics,11.127778,23.748094,12.620316


In [50]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me1_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/amazon/gfk/me1_3000.csv
Resultados guardados.
